In [1]:
%pip install --upgrade pip
%pip install farm-haystack

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
import os
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
import joblib

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

/Users/ezanaaimero/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-10 11:59:34.662333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0


In [4]:
doc_dir = "../data/Bible_trained5"

In [5]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)



INFO - haystack.pipelines.base -  It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing:   0%|                                  | 0/429 [00:00<?, ?docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  We found one or more sentences whose word count is higher than the split length.
WARNING - haystack.nodes.preprocessor.preprocessor -  Document dbca710f7b349551a9ed9441e131a181 is 27342 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Preprocessing:  27%|██████                 | 114/429 [00:00<00:00, 857.06docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  Document 78f8531af4f257b084a3c7c611d5cf61 is 15360 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query t

{'documents': [<Document: {'content': 'ምዕራፍ 10\nእኔም ራሴ ጳውሎስ፥ በእናንተ ዘንድ ፊት ለፊት ሳለሁ ትሑት የሆንሁ፥ ከእናንተ ግን ብርቅ የምደፍራችሁ፥ በክርስቶስ የዋህነትና ገርነት እመክራችኋለሁ፤\n2 በዓለማዊ ልማድ እንደምንመላለስ በሚቆጥሩን በአንዳንዶች ላይ አምኜ ልደፍር አስባለሁ፥ በዚያ እምነት ግን ከእናንተ ጋር ሆኜ እንዳልደፍር እለምንችኋለሁ።\n3 በሰው ልማድ ምንም እንኳ የምንመላለስ ብንሆን፥ እንደ ሰው ልማድ አንዋጋም፤\n4 የጦር ዕቃችን ሥጋዊ አይደለምና፥ ምሽግን ለመስበር ግን በእግዚአብሔር ፊት ብርቱ ነው፤\n5 የሰውንም አሳብ በእግዚአብሔርም እውቀት ላይ የሚነሣውን ከፍ ያለውን ነገር ሁሉ እናፈርሳለን ለክርስቶስም ለመታዘዝ አእምሮን ሁሉ እንማርካለን፥\n6 መታዘዛችሁም በተፈጸመች ጊዜ አለመታዘዝን ሁሉ ልንበቀል ተዘጋጅተናል።\n7 በፊታችሁ ያለውን ተመልከቱ። ማንም የክርስቶስ መሆኑን ተረድቶ ቢሆን፥ ይህን እንደ ገና በራሱ ይቁጠረው፤ እርሱ የክርስቶስ እንደ ሆነ እኛ ደግሞ እንዲሁ ነን።\n8 ጌታ እናንተን ለማነጽ እንጂ እናንተን ለማፍረስ ያይደለ በሰጠው በሥልጣናችን ከፊት ይልቅ ብመካ እንኳ አላፍርም።\n9 በመልእክቶቼ የማስደነግጣችሁ አይምሰላችሁ።\n10 መልእክቶቹስ ከባድና ኃይለኛ ናቸው፥ ሰውነቱ ግን ሲታይ ደካማ ነው፥ ንግግሩም የተናቀ ነው ይላሉና።\n11 እንዲሁ የሚል ይህን ይቁጠረው፤ በሩቅ ሳለን በመልእክታችን በኩል በቃል እንዳለን፥ በፊቱ ደግሞ ሳለን በሥራ እንዲሁ ነን።\n12 ራሳቸውን ከሚያመሰግኑ ከአንዳንዶች ጋር ራሳችንን ልንቆጥር ወይም ራሳችንን ልናስተያይ አንደፍርምና፤ ነገር ግን እነርሱ ራሳቸውን ከራሳቸው ጋር ሲያመዛዝኑ፥ ራሳቸውንም ከራሳቸው ጋር ሲያስተያዩ፥ አያስተውሉም።\n13 እኛ ግን እግዚአብሔር እ

In [6]:
from haystack.nodes import BM25Retriever
from haystack.nodes import DensePassageRetriever
retriever = BM25Retriever(document_store=document_store)

In [7]:
from haystack.nodes import FARMReader
#*** worked nice in English deepset/roberta-base-squad2
#AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru 
#distilbert-base-cased-distilled-squad
#xlm-roberta-base ----amharic
reader = FARMReader(model_name_or_path="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru", use_gpu=True)



INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru' (XLMRoberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: multilingual
INFO - haystack.modeling.model.language_model -  Loaded 'AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru' (XLMRoberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0


In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [9]:
from haystack.utils import print_answers

#print_answers(
#    prediction,
#    details="medium" ## Choose from `minimum`, `medium`, and `all`
#)



In [10]:
def createVerse2(verse_after_decimal_point):
    if verse_after_decimal_point == 1:
        return str (verse_after_decimal_point) + "-" + str (verse_after_decimal_point + 1)
    else:
        return str (verse_after_decimal_point - 1) + "-" + str (verse_after_decimal_point + 1)
    



def findTriguame(verse_start):
    varse = "empty"
    
    with open('../data/Bible_verse_with_triguame_full.txt', 'r') as file:
        data = file.read()
    triguame_start = 0
    while data[verse_start:verse_start+12] != 'triguame_end':
            #print("TTTTTTTTT ",data[verse_start:verse_start+14])
            if data[verse_start:verse_start+14] == "triguame_start":
                triguame_start = verse_start
            
            verse_start = verse_start + 1
    
    return data[triguame_start+15:verse_start]
    
    

def findVerse2(searchInput):
    print("<<<<<<<<<<<<<<<<<<<",searchInput)
    varse = "empty"
    triguame_result = ""
    with open('../data/Bible_verse_with_triguame_full.txt', 'r') as file:
        data = file.read()
    
    
    searchResult = data.find(searchInput)
    tabIndex = 0
    endOfVerse = 0
    decimal_point = 0
    if searchResult != -1:
        while data[searchResult:searchResult+11] != 'verse_start':
            if data[searchResult:searchResult+12] == 'verse_number':
                tabIndex = searchResult
                endOfVerse = tabIndex
            if data[searchResult:searchResult+11] == 'verse_point':
                decimal_point = searchResult
            searchResult = searchResult - 1
        #verse string and number    
        varse_number = data[searchResult+12:decimal_point]

        verse_after_decimal_point = int(data[decimal_point+11:tabIndex-1])
        #first verse
        searchResult = searchResult - 1
        endOfFirstVerse = searchResult
        
        first_verse_end = 0
        while data[searchResult:searchResult+12] != 'verse_number':
            #print("TTTTTTTTT ",data[searchResult:searchResult+9])
            if data[searchResult:searchResult+9] == "verse_end":
                first_verse_end = searchResult
            searchResult = searchResult - 1
        varse_sentence_1 = data[searchResult+15: first_verse_end-1]    
        
        #second Verse  
        while data[endOfVerse:endOfVerse+9] != 'verse_end':
            endOfVerse = endOfVerse + 1    
            
        varse_sentence_2 = data[tabIndex+15:endOfVerse-11]
        verse_start = tabIndex
        endOfVerse = endOfVerse+1
        second_tab = 0
        #third verse
        while data[endOfVerse:endOfVerse+9] != 'verse_end':
            if data[endOfVerse:endOfVerse+12] == 'verse_number':
                second_tab = endOfVerse
            endOfVerse = endOfVerse + 1
            
        varse_sentence_3 = data[second_tab+14:endOfVerse-1]
        varse = varse_number + "÷" + createVerse2(verse_after_decimal_point) + ": "+ varse_sentence_1+" \n"+varse_sentence_2 + "\n" + varse_sentence_3
        triguame_result = findTriguame(verse_start)
        triguame_result = triguame_result.replace("\n","")
    return varse,triguame_result


In [11]:
app = Flask(__name__)
api = Api(app)
class MakePrediction(Resource):
    try:
        
        @staticmethod
        def post():
            posted_data = request.get_json()
            user_question = posted_data['question']
            retriver_top_k = 10
            reader_top_k = 5
            prediction = pipe.run(
            query=user_question,
            params={
                "Retriever": {"top_k": retriver_top_k},
                "Reader": {"top_k": reader_top_k}
                })
            
            #=============================
            print_answers(
                prediction,
                details="medium" ## Choose from `minimum`, `medium`, and `all`
            )

            #=============================


            text = ""
            answerList = []
            verseList = []
            triguameList = []
            for x in range(reader_top_k):
                new_context = prediction['answers'][x].context 
                context_list = new_context.split('\n')
                context_list.sort(key=len, reverse=True)
                print(f"Answer: '{prediction['answers'][x].answer}', score: {round(prediction['answers'][x].score, 4)}")

                #add to answersList that will returned by API
                answerList.append(prediction['answers'][x].answer)
                outputVarse = ""
                outputTriguame = ""
                for sentence in context_list:
                    sentence = sentence[3:]
                    #print(sentence,"<====")
                    if len(sentence) < 3: continue
                    #outputVarse = findVerse2(sentence)
                    #print(sentence,"<========")
                    sentence = sentence.replace("\n","")
                    #sentence = sentence.replace("»","")
                    #sentence = sentence.replace("«","")
                    outputVarse,outputTriguame = findVerse2(sentence)

                    if outputVarse != "":
                        break
                        
                outputVarse = outputVarse.replace('\t', '')
                print("\t",outputVarse)
                text = text + new_context
                print()
                print("\t«ከትርጓሜ መጻሕፍት»")
                print()
                print(outputTriguame[:400])

                #add triguame to triguame list that will be returned to api
                verseList.append(outputVarse)
                triguameList.append(outputTriguame[:400])
                print()
                print("===============================================================")

            return jsonify({
                'answers': answerList,
                'verses': verseList,
                'triguames':triguameList
            })
    except:
        print("api error main")
try:
    api.add_resource(MakePrediction, '/predict')
except:
    print("api erreor")

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO - werkzeug -  WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO - werkzeug -  Press CTRL+C to quit
